# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [4]:
def priest(patients_sex, patients_age, patients_resp, patients_os, patients_heart, patients_bp, patients_temp, patients_alert, patients_inspo, patients_status):
    """(str, int, int, int, int, float, float, str, str, str) -> (float)
    Compute a risk percentage using the algorithm from MDCalc portal.

    >>> priest('Male', 81, 27, 0.91, 40, 111, 36.1, 'Alert', 'AIR', 'Unrestricted normal activity')
    0.47

    >>> priest('MALE', 66, 12, 0.97, 132, 80, 34.5, 'Alert', 'Supplemental oxygen', 'Bed/chair bound, no self-care')
    0.59

    >>> priest('MALE', 16, 8, 0.89, 51, 169, 38.9, 'Confused or not alert', 'Supplemental oxygen', 'Unrestricted normal activity')
    0.46
    """
    risk_count = 0
    last_risk = 0

    # 'casefold()' method is stronger(https://www.w3schools.com/python/ref_string_casefold.asp)
    patients_status = patients_status.casefold()
    patients_sex = patients_sex.casefold()
    patients_alert = patients_alert.casefold()
    patients_inspo = patients_inspo.casefold()

    # check
    if patients_status not in ["unrestricted normal activity", "limited strenuous activity, can do light activity",
                               "limited activity, can self-care", "limited self-care", "bed/chair bound, no self-care"]:
        return None
    if patients_sex not in ["male", "female"]:
        return None
    if patients_alert not in ["alert", "confused or not alert"]:
        return None
    if patients_inspo not in ["air", "supplemental oxygen"]:
        return None

    # Sex
    if patients_sex == "male":
        risk_count += 1

    # Age
    if patients_age > 80:
        risk_count += 4
    elif patients_age >= 66:
        risk_count += 3
    elif patients_age >= 50:
        risk_count += 2

    # Respiratory rate
    if patients_resp < 9:
        risk_count += 3
    if patients_resp > 24:
        risk_count += 3
    elif 21 <= patients_resp <= 24:
        risk_count += 2
    elif 9 <= patients_resp <= 11:
        risk_count += 1

    # Oxygen saturation
    if patients_os < 0.92:
        risk_count += 3
    elif patients_os <= 0.93:
        risk_count += 2
    elif patients_os <= 0.95:
        risk_count += 1

    # Heart rate
    if patients_heart > 130:
        risk_count += 3
    elif patients_heart < 41:
        risk_count += 3
    elif 50 >= patients_heart >= 41 or 110 >= patients_heart >= 91:
        risk_count += 1
    elif 130 >= patients_heart >= 111:
        risk_count += 2

    # Systolic BP
    if patients_bp < 91 or patients_bp > 219:
        risk_count += 3
    elif 91 <= patients_bp <= 100:
        risk_count += 2
    elif 101 <= patients_bp <= 110:
        risk_count += 1

    # Temperature
    if patients_temp < 35.1:
        risk_count += 3
    elif 35.1 <= patients_temp <= 36.0:
        risk_count += 1
    elif 38.1 <= patients_temp <= 39.0:
        risk_count += 1
    elif 36.1 <= patients_temp <= 38.0:
        risk_count += 0
    else:
        risk_count += 2

    # Alertness
    if patients_alert == "confused or not alert":
        risk_count += 3

    # Inspired oxygen
    if patients_inspo == "supplemental oxygen":
        risk_count += 2

    # Performance status
    if patients_status == "bed/chair bound, no self-care":
        risk_count += 4
    elif patients_status == "limited strenuous activity, can do light activity":
        risk_count += 1
    elif patients_status == "limited activity, can self-care":
        risk_count += 2
    elif patients_status == "limited self-care":
        risk_count += 3

    # Risk percentage
    risk_map = {0: 0.01, 1: 0.01, 2: 0.02, 3: 0.02, 4: 0.03, 5: 0.09, 6: 0.15, 7: 0.18, 8: 0.22, 9: 0.26, 10: 0.29,
                    11: 0.34, 12: 0.38, 13: 0.46, 14: 0.47, 15: 0.49, 16: 0.55}

    if risk_count in risk_map:
        last_risk += risk_map[risk_count]
    elif 17 <= risk_count <= 25:
        last_risk += 0.59
    else:
        last_risk += 0.99

    # Return
    return last_risk

In [5]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('Male', 81, 27, 0.91, 40, 111, 36.1, 'Alert', 'AIR', 'Unrestricted normal activity')
Expecting:
    0.47
ok
Trying:
    priest('MALE', 66, 12, 0.97, 132, 80, 34.5, 'Alert', 'Supplemental oxygen', 'Bed/chair bound, no self-care')
Expecting:
    0.59
ok
Trying:
    priest('MALE', 16, 8, 0.89, 51, 169, 38.9, 'Confused or not alert', 'Supplemental oxygen', 'Unrestricted normal activity')
Expecting:
    0.46
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [6]:
import requests

def find_hospital(age, sex, priest):
  """(int, str, float) -> (str)
  The function retrieves the name of the hospital by accessing a web service.

  >>> find_hospital(30, 'male', 0.10)
  'Southwest Hospital and Medical Center'

  >>> find_hospital(30, 'female', 0.20)
  'Select Specialty Hospital - Northeast Atlanta'

  >>> find_hospital(68, 'male', 0.30)
  'Wesley Woods Geriatric Hospital'
  """
  # REST web service
  api = 'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={}&sex={}&risk_pct={}'.format(age, sex, priest)
  request = requests.get(api)

  hospital_json = request.json()
  hospital = hospital_json.get('hospital')

  # Check
  if hospital_json:
    return hospital
  else:
    return None

In [7]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(30, 'male', 0.10)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(30, 'female', 0.20)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(68, 'male', 0.30)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [8]:
import requests
import json

def get_address(hospital):
  """(str) -> (str)
   Function will acquire the hospital's address from json

   >>> get_address('PHOEBE SUMTER MEDICAL CENTER')
   '126 HIGHWAY 280 W'

   >>> get_address('ATHENS REGIONAL MED CENTER')
   '1199 PRINCE AVENUE'

   >>> get_address('ST MARYS HEALTH CARE SYSTEM')
   '1230 BAXTER STREET'
  """

  # Load
  with open('ga_hospitals.json') as ga:
    hospital_json = json.load(ga)

  address = hospital_json[hospital]['ADDRESS']

  # Check
  if hospital in hospital_json:
    return address
  else:
    return None

## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [9]:
import pandas as pd

def process_people(path):
  """(path) -> (dict)
   Function wil use path provided and returns a list of dictionaries.
  """
  last_return = {}

  # Load
  csv = pd.read_csv(path, sep='|')

  # Loop using 'iterrows()'(https://www.w3schools.com/python/pandas/ref_df_iterrows.asp)
  for index, row in csv.iterrows():
    patients_code = row['patient']
    patients_sex = row['sex']
    patients_age = row['age']
    patients_resp = row['breath']
    patients_os = row['o2 sat']
    patients_heart = row['heart']
    patients_bp = row['systolic bp']
    patients_temp = row['temp']
    patients_alert = row['alertness']
    patients_inspo = row['inspired']
    patients_status = row['status']

    calculate_risk = priest(patients_sex, patients_age, patients_resp, patients_os, patients_heart, patients_bp, patients_temp, patients_alert, patients_inspo, patients_status)
    hospital_name = find_hospital(patients_age, patients_sex, calculate_risk)
    hospital_address = get_address(hospital_name.upper())

    last_return[patients_code] = [
        patients_sex, patients_age, patients_resp, patients_os, patients_heart, patients_bp, patients_temp, patients_alert, patients_inspo, patients_status, calculate_risk, hospital_name, hospital_address
    ]

  return last_return

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [10]:
# Final check
# Load
with open('people_results.json') as people:
  final_check = json.load(people)

# Correct or Incorrect
if process_people('people.psv') == final_check:
    print('Correct')
else:
    print('Incorrect')


Correct


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---